In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import tensorflow as tf
import math
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from random import sample
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from  tensorflow.keras.metrics import AUC, Precision, Recall, Accuracy
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from utils import plot_metrics
from utils import F1Score


In [2]:
img_height = img_width = 300

def resize_img(img):
    try:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    except:
        print('error in resizing')
        img1 = isinstance(img, type(None))
        print('Does image is none: ', img1)
        print(img.shape)
    return cv2.resize(img, (img_height, img_width))

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img.astype(np.uint8))

def canny_cropping(img):
    convert_img = np.array(img, dtype=np.uint8)

    gray = cv2.cvtColor(convert_img, cv2.COLOR_RGB2GRAY)


    ave_brightness = math.floor(np.average(gray))
    min_pixel = min(gray.flatten())

    edges = cv2.Canny(gray, min_pixel, ave_brightness)
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        x, y, w, h = cv2.boundingRect(edges)
        gray = gray[y:y+h, x:x+w]
        break

    return gray

def preprocessing_without_clahe(img):
    cropped = canny_cropping(img)
    return resize_img(cropped)

def preprocessing_with_clahe(img):
    cropped = canny_cropping(img)
    clahe = apply_clahe(cropped)
    return resize_img(clahe)

In [3]:
data_directory = 'dataset'

train_img = pd.read_csv(os.path.join(data_directory, 'MURA-v1.1/train_image_paths.csv'), names=['path'])
test_img = pd.read_csv(os.path.join(data_directory, 'MURA-v1.1/test_image_paths.csv'), names=['path'])


In [4]:
test_img['label'] = test_img['path'].map(lambda x: '1' if 'positive' in x else '0')
test_class_frequency = pd.DataFrame(test_img['label'].value_counts())

In [6]:
test_img['study_type'] = test_img['path'].map(
    lambda x: x.split('/')[2]
)

test_img

,path,label,study_type
0,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
1,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
2,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
3,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
4,MURA-v1.1/test/XR_WRIST/patient11186/study1_po...,1,XR_WRIST
...,...,...,...
3192,MURA-v1.1/test/XR_FINGER/patient11967/study1_n...,0,XR_FINGER
3193,MURA-v1.1/test/XR_FINGER/patient11967/study1_n...,0,XR_FINGER
3194,MURA-v1.1/test/XR_FINGER/patient11738/study1_n...,0,XR_FINGER
3195,MURA-v1.1/test/XR_FINGER/patient11738/study1_n...,0,XR_FINGER


In [68]:
def plot_radiograph(img, title):
    plt.imshow(img['image'])
    plt.tight_layout()
    plt.suptitle(img['label'],  y=1.01)
    plt.axis('off')
    plt.show()

def map_label(label):
    if label == '1':
        return 'normal'
    elif label == '0':
        return 'abnormal'
    else:
        return 'none'

def get_sample_img(test_img, part):
    parts = pd.DataFrame(test_img[test_img.study_type == part])
    
    abnormal = parts[parts.label == '1'].sample(1)
    normal = parts[parts.label == '0'].sample(1)

    normal_path = normal['path'].values[0]
    abnormal_path = abnormal['path'].values[0]

    return [cv2.imread(os.path.join(data_directory, normal_path), 1), cv2.imread(os.path.join(data_directory, abnormal_path), 1)]

    

In [52]:
STUDY_TYPES = [
    'XR_SHOULDER',
    'XR_ELBOW',
    'XR_HUMERUS',
    'XR_HAND',
    'XR_WRIST',
    'XR_FOREARM',
    'XR_FINGER',
]

In [71]:
path_to_save_img = 'images/test_cases'

for study in STUDY_TYPES:
    normal, abnormal = get_sample_img(test_img, study)

    # Save raw images
    cv2.imwrite(f'{path_to_save_img}/{study}_normal_raw.png', normal)
    cv2.imwrite(f'{path_to_save_img}/{study}_abnormal_raw.png', abnormal)

    # Save preprocessd image without clahe
    cv2.imwrite(f'{path_to_save_img}/{study}_normal_without_clahe.png', preprocessing_without_clahe(normal))
    cv2.imwrite(f'{path_to_save_img}/{study}_abnormal_without_clahe.png', preprocessing_without_clahe(abnormal))

    # Save preprocessd image with clahe
    cv2.imwrite(f'{path_to_save_img}/{study}_normal_with_clahe.png', preprocessing_with_clahe(normal))
    cv2.imwrite(f'{path_to_save_img}/{study}_abnormal_with_clahe.png', preprocessing_with_clahe(abnormal))



563
465
288
460
659
301
461
